<a href="https://colab.research.google.com/github/vadhri/ai-notebook/blob/main/he/fhe_secure_voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Multi-option secure voting system

A sample implementation of secure voting system with the following parameters.

- There are 10 options (e.g., candidates or choices 0–9).
- Each participant chooses one option (say, 3 or 7).
- Each participant encrypts their choice using BFV (homomorphic encryption).
- The system stores ciphertexts in a Pandas DataFrame (so you can inspect, store, or later aggregate).

A homomorphic aggregation step tallies encrypted votes across all participants, and decrypts the total votes per option.

In [2]:
# Optional
!sudo apt-get install git build-essential cmake python3 python3-dev python3-pip
%cd /content
!rm -rf /content/SEAL-Python

# # Get the repository or download from the releases
!git clone https://github.com/Huelse/SEAL-Python.git
%cd SEAL-Python

# # Install dependencies
!pip3 install numpy pybind11

# Init the SEAL and pybind11
!git submodule update --init --recursive
# Get the newest repositories (dev only)
# !git submodule update --remote

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.15).
python3 is already the newest version (3.10.6-1~22.04.1).
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-pip is already the newest version (22.0.2+dfsg-1ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
/content
Cloning into 'SEAL-Python'...
remote: Enumerating objects: 1642, done.
remote: Counting objects: 100% (270/270), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 1642 (delta 185), reused 204 (delta 167), pack-reused 1372 (from 1)
Receiving objects: 100% (1642/1642), 8.68 MiB | 17.42 MiB/s, done.
Resolving deltas: 100% (890/890), done.
/content/SEAL-Python
Submodule 'SEAL' (https://github.com/microsoft/SEAL.git) registered for path 'SEAL

In [3]:
# Build the SEAL lib without the msgsl zlib and zstandard compression
%cd SEAL
!cmake -S . -B build -DSEAL_USE_MSGSL=OFF -DSEAL_USE_ZLIB=OFF -DSEAL_USE_ZSTD=OFF
!cmake --build build
%cd ..

/content/SEAL-Python/SEAL
-- Build type (CMAKE_BUILD_TYPE): Release
-- The CXX compiler identification is GNU 11.4.0
-- The C compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Microsoft SEAL debug mode: OFF
-- SEAL_USE_CXX17: ON
-- SEAL_BUILD_DEPS: ON
-- SEAL_USE_MSGSL: OFF
-- SEAL_USE_ZLIB: OFF
-- SEAL_USE_ZSTD: OFF
-- SEAL_USE_INTEL_HEXL: OFF
-- BUILD_SHARED_LIBS: OFF
-- SEAL_THROW_ON_TRANSPARENT_CIPHERTEXT: ON
-- SEAL_USE_GAUSSIAN_NOISE: OFF
-- SEAL_DEFAULT_PRNG: Blake2xb
-- SEAL_AVOID_BRANCHING: OFF
-- x86intrin.h - found
-- SEAL_USE_INTRIN: ON
-- Performing Test SEAL_MEMSET_S_FOUND
-- Per

In [4]:
#Run the setup.py, the dynamic library will be generated in the current directory
!python3 setup.py build_ext -i

running build_ext
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/include/python3.12 -c flagcheck.cpp -o flagcheck.o -std=c++17
building 'seal' extension
creating build/temp.linux-x86_64-cpython-312/src
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -DVERSION_INFO=4.0.0 -I/usr/include/python3.12 -Ipybind11/include -ISEAL/native/src -ISEAL/build/native/src -I/usr/local/lib/python3.12/dist-packages/pybind11/include -I/usr/include/python3.12 -c src/wrapper.cpp -o build/temp.linux-x86_64-cpython-312/src/wrapper.o -std=c++17 -fvisibility=hidden -g0 -std=c++17
creating build/lib.linux-x86_64-cpython-312
x86_64-linux-gnu-g++ -fno-strict-overflow -Wsign-compare -DNDEBUG -g -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -share

In [5]:
!pip install .

Processing /content/SEAL-Python
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for seal: filename=seal-4.0.0-cp312-cp312-linux_x86_64.whl size=611226 sha256=c6ec938ac9945b7a81ee5c67c83a83db525ca377c977d8f93f6592e0def88c3b
  Stored in directory: /root/.cache/pip/wheels/8f/9b/a2/8f17830c264fc4f685de56341ecac4c286e5d05e24b9aae978
Successfully built seal
  Attempting uninstall: seal
    Found existing installation: seal 4.0.0
    Uninstalling seal-4.0.0:
      Successfully uninstalled seal-4.0.0


In [6]:
from seal import *
import numpy as np
import pandas as pd
import io

parms = EncryptionParameters(scheme_type.bfv)
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(PlainModulus.Batching(poly_modulus_degree, 20))
context = SEALContext(parms)

keygen = KeyGenerator(context)
secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
encryptor = Encryptor(context, public_key)
decryptor = Decryptor(context, secret_key)
evaluator = Evaluator(context)
encoder = BatchEncoder(context)

slot_count = encoder.slot_count()
print(f"Slot count: {slot_count}")

Slot count: 8192


In [10]:

def one_hot_encode(choice, num_options=10):
    vec = [0] * num_options
    vec[choice] = 1
    return vec

def encrypt_vote(choice, num_options=10):
    one_hot = one_hot_encode(choice, num_options)
    vec = np.zeros(slot_count, dtype=np.int64)
    vec[:num_options] = np.array(one_hot, dtype=np.int64)

    pt = encoder.encode(vec)
    ct = encryptor.encrypt(pt)
    return ct

def decrypt_tally(ct, num_options):
    pt = Plaintext()
    decryptor.decrypt(ct, pt)
    decoded = encoder.decode(pt)
    return [int(decoded[i]) for i in range(num_options)]


def add_ciphertexts(ct_list):
    if not ct_list:
        return None

    total = ct_list[0]

    for ct in ct_list[1:]:
        evaluator.add_inplace(total, ct)

    return total


In [13]:
from collections import Counter
num_options = 10
participants = 1500

choices = np.random.randint(0, num_options, participants)

# top 5 choices
top_5_choices = Counter(choices).most_common(5)
print(top_5_choices)

# Encrypt and store in a DataFrame
rows = []
for pid, choice in enumerate(choices):
    ct = encrypt_vote(choice, num_options)
    rows.append({
        "Participant_ID": pid,
        "Choice": choice,
        "EncryptedChoice": ct
    })

df = pd.DataFrame(rows)
print("\nEncrypted votes DataFrame:")
print(df[["Participant_ID", "Choice", "EncryptedChoice"]])

[(np.int64(2), 172), (np.int64(3), 161), (np.int64(4), 156), (np.int64(9), 154), (np.int64(8), 151)]

Encrypted votes DataFrame:
      Participant_ID  Choice                             EncryptedChoice
0                  0       2  <seal.Ciphertext object at 0x7f4e5ccfec70>
1                  1       9  <seal.Ciphertext object at 0x7f4e5d6cd9f0>
2                  2       8  <seal.Ciphertext object at 0x7f4e5ccfc4b0>
3                  3       3  <seal.Ciphertext object at 0x7f4e5cbf6cf0>
4                  4       9  <seal.Ciphertext object at 0x7f4e5cbf6730>
...              ...     ...                                         ...
1495            1495       7  <seal.Ciphertext object at 0x7f4e2ba2f770>
1496            1496       6  <seal.Ciphertext object at 0x7f4e2ba2f7f0>
1497            1497       7  <seal.Ciphertext object at 0x7f4e2ba2f870>
1498            1498       3  <seal.Ciphertext object at 0x7f4e2ba2f8f0>
1499            1499       6  <seal.Ciphertext object at 0x7f4e2ba2f

In [14]:
ct_sum = add_ciphertexts(df["EncryptedChoice"].tolist())

# Decrypt tally
tally = decrypt_tally(ct_sum, num_options)
print("\nFinal Tally (per option):")
for i, count in enumerate(tally):
    print(f"Option {i}: {count} votes")

# For cross-check (non-secure verification)
expected_counts = np.bincount(choices, minlength=num_options)
print("\nExpected (plaintext check):", expected_counts.tolist())

print(f"Maximum counts : ", np.argmax(expected_counts))


Final Tally (per option):
Option 0: 137 votes
Option 1: 146 votes
Option 2: 172 votes
Option 3: 161 votes
Option 4: 156 votes
Option 5: 130 votes
Option 6: 147 votes
Option 7: 146 votes
Option 8: 151 votes
Option 9: 154 votes

Expected (plaintext check): [137, 146, 172, 161, 156, 130, 147, 146, 151, 154]
Maximum counts :  2
